In [1]:
# dependencies
import pandas as pd

import sqlalchemy

from sqlalchemy import create_engine, MetaData, inspect, func
from sqlalchemy.ext.declarative import declarative_base
from datetime import datetime

from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date

In [2]:
Base = declarative_base()

In [3]:
# Create Database Connection
engine = create_engine('sqlite:///hawaii.sqlite')
Base.metadata.create_all(engine)

In [4]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [5]:
# Use `_base` from SQLAlchemy to model the measurements table as an ORM class
class Measurements(Base):
    __tablename__ = 'measurements'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Date)
    prcp = Column(Float)  
    tobs = Column(Float)

In [6]:
# Use `_base` from SQLAlchemy to model the stations table as an ORM class
class Stations(Base):
    __tablename__ = 'stations'
    
    id = Column(Integer, primary_key=True)
    station = Column(Text)
    name = Column(Text)
    latitude= Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)


In [7]:
#will get the column names and type using inspect 
inspector = inspect(engine)
inspector.get_table_names()
# Get a list of column names and types
columns = inspector.get_columns('Measurements')
for c in columns:
    print(c['name'], c["type"])
# columns

id INTEGER
station TEXT
date DATE
prcp FLOAT
tobs FLOAT


In [8]:
# check table is in db
conn.execute('SELECT * FROM Measurements LIMIT 5').fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-07', 0.06, 70.0)]

In [9]:
#read csv and convert to df
measurement_df = pd.read_csv("clean_hawaii_measurements.csv")
measurement_df.shape

(18103, 4)

In [10]:
station_df = pd.read_csv("clean_hawaii_station.csv")
station_df.shape

(9, 5)

In [11]:
measurement_df['date'] = pd.to_datetime(measurement_df['date'], format = '%Y-%m-%d')

In [12]:
# Data is just a list of dictionaries that represent each row of data
df_measurement = measurement_df.to_dict(orient='records')
df_measurement[0]

{'date': Timestamp('2010-01-01 00:00:00'),
 'prcp': 0.08,
 'station': 'USC00519397',
 'tobs': 65}

In [13]:
df_station = station_df.to_dict(orient="records")
df_station[0]

{'elevation': 3.0,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [14]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [15]:
# Save the reference to the measurements table as a variable called m_table
m_table = sqlalchemy.Table('measurements', metadata, autoload=True)


In [16]:
# Use table.delete() to remove any pre-existing data.
# convenience function so that you can re-run multiple times.
conn.execute(m_table.delete())

In [17]:
# data populated to measurements table 
conn.execute(m_table.insert(), df_measurement)

In [18]:
# fetching the first 5 records.
conn.execute("select * from measurements limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-07', 0.06, 70.0)]

In [19]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [20]:
Base.metadata.create_all(engine)
conn = engine.connect()

In [21]:
s_table= sqlalchemy.Table("stations", metadata, autoload=True)

In [22]:
conn.execute(s_table.delete())
conn.execute(s_table.insert(), df_station)

In [23]:
conn.execute("select * from stations").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 (6, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 (7, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9),
 (8, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 (9, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]